# Librerias

In [1]:
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings("ignore")

# Variables

In [3]:
tema_azul = ['#0000CC','#3366FF','#99CCFF']
tema_rojo = ['#CC0000','#FF3333','#FF9999']
tema_verde = ['#009900','#33AA33','#55BB55']
tema_negro = ['#000000','#333333','#555555']

# Funciones

# Importacion de datos

## Lectura de Origenes

### Datos Generales de ventas desagregados

In [4]:
vent_full = pd.read_excel('.\Data\Descarga_VENTAS_POR_LOCALES_NB_050923.xlsx',skiprows=1)
vent_full = vent_full.drop(columns=['Valores'
                                    , 'Centro Comercial'
                                    , 'Identificacion'
                                    , 'Contrato'
                                    , 'Estado'
                                    , 'Faltan', 'Cero'
                                    , 'Calculo'])
vent_full = vent_full.astype({'Nombre Comercial': 'category'
                                ,'Local': 'category'
                                ,'Rubro': 'category'
                                ,'Periodo': 'datetime64[ns]'
                                , 'Rubros': 'category'
                                })
vent_full.head()

,Local,Superficie,Razón Social,Nombre Comercial,Rubro,Periodo,Rubros,Cantidad,Importe
0,1,101.8,MUEBLES EXPRESS S.R.L.,DELLACASA,DECORACION,2022-07-01,GENERAL,27.0,4379919.00
1,101,211.0,AVE CAESAR S.R.L.,SENSU,GASTRONOMIA,2022-07-01,GENERAL,2980.0,7630451.01
2,102,204.5,LAS BLONDAS SA,RAPSODIA,INDUMENTARIA FEMENINA,2022-07-01,GENERAL,1421.0,28705514.00
3,103/A,81.3,KOWZEF S.A.,HERENCIA ARGENTINA,INDUMENTARIA MASCULINA,2022-07-01,GENERAL,779.0,12426009.52
4,103/B,56.6,TEJIDOS LATINOS SA,WANAMA BOYS & GIRLS,INDUMENTARIA NIÑOS,2022-07-01,GENERAL,414.0,4745312.22


### Indice de Precios de Consumo

In [5]:
df_ipc = pd.read_excel('.\Data\Tabla_IPC.xlsx',skiprows=1)
df_ipc.rename(columns={'Fecha': 'fecha', 'Indice': 'ipc'}, inplace = True)
df_ipc.set_index('fecha', inplace=True)
df_ipc['ipc_previo'] = df_ipc['ipc'].shift(1)
df_ipc['ipc_intermensual'] = (df_ipc['ipc'] - df_ipc['ipc_previo']) / df_ipc['ipc']
df_ipc.reset_index(inplace=True)
df_ipc.head()

,fecha,ipc,ipc_previo,ipc_intermensual
0,2016-12-01,100.0000,NaN,NaN
1,2017-01-01,101.5859,100.0000,0.015611
2,2017-02-01,103.6859,101.5859,0.020253
3,2017-03-01,106.1476,103.6859,0.023191
4,2017-04-01,108.9667,106.1476,0.025871


### Dolar Historico

In [6]:
df_usd = pd.read_excel('.\Data\Dolar_Evolución.xlsx',skiprows=0)
df_usd = df_usd.iloc[:, 1:]
df_usd.columns = ['fecha', 'ofi', 'blue']
df_usd['brecha'] = (df_usd['blue']-df_usd['ofi'])/df_usd['ofi']
df_usd.set_index('fecha', inplace=True)
df_usd.head()

,ofi,blue,brecha
fecha,,,
2015-01-01,8.55,13.50,0.578947
2015-02-01,8.63,12.93,0.498262
2015-03-01,8.72,12.52,0.435780
2015-04-01,8.81,12.60,0.430193
2015-05-01,8.90,12.55,0.410112


### Clima

In [7]:
df_clima = pd.read_excel('.\Data\Clima.xlsx',skiprows=0)
df_clima.head()

,fecha,Ventarron,DiasLluvia,LluviaFuerte,DiasNublados,DiasNubladosLeve,DiasNubladosFuerte,TempMinAvg,TempAvg,TempMaxAvg,DiasCalurosos,DiasFrios,DiasTemplados
0,2017-01-01,1,5,0,9,7,3,22.0,26.0,30.0,11,0,12
1,2017-02-01,2,9,2,23,22,9,21.0,25.0,29.0,19,0,23
2,2017-03-01,1,5,2,24,18,7,19.0,23.0,27.0,6,0,29
3,2017-04-01,6,10,2,26,25,7,16.0,18.0,22.0,0,0,30
4,2017-05-01,0,8,3,27,25,15,14.0,16.0,19.0,0,0,31


## Feature Engineering

### Ventas Ajustado por IPC y Dolar

In [8]:
df_vent = vent_full
df_vent = pd.merge(df_vent, df_ipc, left_on = 'Periodo', right_on= 'fecha' , how='outer')
df_vent = pd.merge(df_vent, df_usd, left_on = 'Periodo', right_on= 'fecha' , how='left')
df_vent.dropna(subset=['ipc'], inplace=True)
df_vent.dropna(subset=['blue'], inplace=True)
df_vent['v_total'] = df_vent['Importe'] / (1 + df_vent['ipc_intermensual']) / df_vent['blue']

### Ventas Segmentadas por Serie de Interes y Rubros

In [10]:
df_vent['v_retail'] = df_vent[(df_vent['Razón Social'] != 'GD WORK SA') & (df_vent['Rubro'] != 'SUPERMERCADO')]['v_total']
df_vent['v_super'] = df_vent[df_vent['Rubro'] == 'SUPERMERCADO']['v_total']
df_vent['v_cowork'] = df_vent[df_vent['Razón Social'] == 'GD WORK SA']['v_total']

df_vent['ru_indFemenina'] = df_vent[df_vent['Rubro'] == 'INDUMENTARIA FEMENINA']['v_total']
df_vent['ru_gastronomia'] = df_vent[df_vent['Rubro'] == 'GASTRONOMIA']['v_total']
df_vent['ru_farmacia'] = df_vent[df_vent['Rubro'] == 'FARMACIA']['v_total']
df_vent['ru_varios'] = df_vent[df_vent['Rubro'] == 'VARIOS']['v_total']
df_vent['ru_indMasculina'] = df_vent[df_vent['Rubro'] == 'INDUMENTARIA MASCULINA']['v_total']
df_vent['ru_indInformal'] = df_vent[df_vent['Rubro'] == 'INDUMENTARIA INFORMAL']['v_total']
df_vent['ru_electronica'] = df_vent[df_vent['Rubro'] == 'ELECTRONICA']['v_total']
df_vent['ru_otros'] = df_vent[~df_vent['Rubro'].isin(['INDUMENTARIA FEMENINA'
                                                        ,'GASTRONOMIA'
                                                        ,'FARMACIA'
                                                        ,'VARIOS'
                                                        ,'INDUMENTARIA MASCULINA'
                                                        ,'INDUMENTARIA INFORMAL'
                                                        ,'ELECTRONICA'])]['v_total']

In [11]:
df_vent = df_vent.groupby(['Periodo']).agg({
    'v_total': 'sum'
    ,'v_super': 'sum'
    ,'v_retail': 'sum'
    ,'v_cowork': 'sum'
    ,'ru_indFemenina': 'sum'
    ,'ru_gastronomia': 'sum'
    ,'ru_farmacia': 'sum'
    ,'ru_varios': 'sum'
    ,'ru_indMasculina': 'sum'
    ,'ru_indInformal': 'sum'
    ,'ru_electronica': 'sum'
    ,'ru_otros': 'sum'
    })
df_vent.reset_index(inplace=True)
df_vent.rename(columns={'Periodo': 'mes'}, inplace = True)
df_vent

,mes,v_total,v_super,v_retail,v_cowork,ru_indFemenina,ru_gastronomia,ru_farmacia,ru_varios,ru_indMasculina,ru_indInformal,ru_electronica,ru_otros
0,2016-12-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
1,2017-01-01,7.401473e+06,3.058102e+06,4.343371e+06,0.000000,7.186022e+05,1.076861e+06,479164.831296,731672.266772,120884.220546,264672.940183,464794.258830,3.544821e+06
2,2017-02-01,7.413210e+06,3.147924e+06,4.265286e+06,0.000000,8.063825e+05,1.108259e+06,407068.880560,672479.413925,205467.664854,210586.652541,363576.284565,3.639390e+06
3,2017-03-01,8.561343e+06,3.475063e+06,5.086280e+06,0.000000,1.076153e+06,1.241118e+06,551893.797198,811202.446927,118710.958106,269561.074032,442757.427447,4.049947e+06
4,2017-04-01,8.884078e+06,3.659147e+06,5.224931e+06,0.000000,1.077838e+06,1.300485e+06,526801.868566,967742.975249,146551.499332,258764.502707,373642.734297,4.232251e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,2023-04-01,5.910126e+06,2.059910e+06,3.819893e+06,30321.827254,9.249395e+05,6.619002e+05,440751.055377,498946.606093,260685.684733,153833.867548,196125.283119,2.772943e+06
77,2023-05-01,5.211214e+06,1.764048e+06,3.424183e+06,22983.295128,8.121483e+05,5.703279e+05,419499.758580,458201.487629,226509.169802,143074.753070,133549.993396,2.447903e+06
78,2023-06-01,5.473887e+06,1.649331e+06,3.795262e+06,29294.712179,7.983353e+05,5.944483e+05,460320.093298,464104.037286,393757.245892,205817.939180,207570.321468,2.349534e+06
79,2023-07-01,5.129018e+06,1.416351e+06,3.685180e+06,27487.146695,6.940928e+05,6.446823e+05,497739.796053,548744.548419,181650.730811,202285.555983,216609.088977,2.143213e+06


### Features de Fechas

In [13]:
df_vent['quarter'] = df_vent['mes'].dt.quarter
df_vent['month'] = df_vent['mes'].dt.month
df_vent['year'] = df_vent['mes'].dt.year

### Features de Tipo de Cambio

In [14]:
df_vent = pd.merge(df_vent, df_usd, left_on='mes', right_on='fecha', how='left')

### Features de Clima

In [15]:
df_vent = pd.merge(df_vent, df_clima, left_on='mes', right_on='fecha', how='left')

### Features de IPC

In [16]:
df_vent = pd.merge(df_vent, df_ipc, left_on='mes', right_on='fecha', how='left')

In [17]:
df_vent.drop('fecha_x', axis=1, inplace=True)
df_vent.drop('fecha_y', axis=1, inplace=True)

### Lags

In [19]:
for i in [1,12]:
    df_vent[f'v_total_lag{i}'] = df_vent['v_total'].shift(periods=i)

for i in [1,12]:
    df_vent[f'v_retail_lag{i}'] = df_vent['v_retail'].shift(periods=i)

for i in [1,12]:
    df_vent[f'v_super_lag{i}'] = df_vent['v_super'].shift(periods=i)

for i in [1,12]:
    df_vent[f'v_cowork_lag{i}'] = df_vent['v_cowork'].shift(periods=i)


### Interacciones

In [20]:
df_vent['int_ret_sup'] = df_vent['v_retail'] * df_vent['v_super']
df_vent['int_ret_cow'] = df_vent['v_retail'] * df_vent['v_cowork']
df_vent['int_sup_cow'] = df_vent['v_super'] * df_vent['v_cowork']

In [21]:
df_vent['porc_retail'] = df_vent['v_retail'] / df_vent['v_total']
df_vent['porc_super'] = df_vent['v_super'] / df_vent['v_total']
df_vent['porc_cowork'] = df_vent['v_cowork'] / df_vent['v_total']

In [22]:
df_vent['int_temp_lluvia'] = df_vent['TempAvg'] * df_vent['DiasLluvia']
df_vent['int_temp_nublado'] = df_vent['TempAvg'] * df_vent['DiasNublados']
df_vent['int_temp_viento'] = df_vent['TempAvg'] * df_vent['Ventarron']
df_vent['int_temp_viento_lluvia'] = df_vent['TempAvg'] * df_vent['Ventarron'] * df_vent['DiasLluvia']

In [24]:
df_vent.to_excel('datos.xlsx', index=False)